In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
# Input Data Set #
Data ="BostonHousing"

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,100)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["TreeEnsembleQBCFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "rashomon_bound_adder": [0.01, 0.025, 0.05],
                       "Type": ["Classification"]}


# Create Parameter Vector

In [3]:
# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Filter #
IndexDrop = ParameterVector[(ParameterVector["ModelType"] == "RandomForestClassificationFunction") & (ParameterVector["UniqueErrorsInput"]== 1)].index
ParameterVector.drop(IndexDrop , inplace=True)
ParameterVector.index = range(0,ParameterVector.shape[0])

# Generate JobName #
ParameterVector["JobName"] = (
    "Seed" + ParameterVector["Seed"].astype(str) +
    "_Data" + ParameterVector["Data"].astype(str) +
    "_TP" + ParameterVector["TestProportion"].astype(str) +
    "_CP" + ParameterVector["CandidateProportion"].astype(str) +
    "_ST" + ParameterVector["SelectorType"].astype(str) +
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_NE" + ParameterVector["n_estimators"].astype(str) +
    "_Reg" + ParameterVector["regularization"].astype(str) +
    "_RBA" + ParameterVector["rashomon_bound_adder"].astype(str)
    )

# Output Name #
ParameterVector["JobName"] = ParameterVector["JobName"].str.replace("Function", "", regex=False)
ParameterVector["Output"] =  ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

# Save Parameter Vector

In [4]:
# Save 
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)

In [5]:
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,rashomon_bound_adder,Type,JobName,Output
0,BostonHousing,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.010,Classification,Seed0_DataBostonHousing_TP0.2_CP0.8_STTreeEnse...,BostonHousing/TreeFarms/Raw/Seed0_DataBostonHo...
1,BostonHousing,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.025,Classification,Seed0_DataBostonHousing_TP0.2_CP0.8_STTreeEnse...,BostonHousing/TreeFarms/Raw/Seed0_DataBostonHo...
2,BostonHousing,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.050,Classification,Seed0_DataBostonHousing_TP0.2_CP0.8_STTreeEnse...,BostonHousing/TreeFarms/Raw/Seed0_DataBostonHo...
3,BostonHousing,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.010,Classification,Seed0_DataBostonHousing_TP0.2_CP0.8_STTreeEnse...,BostonHousing/TreeFarms/Raw/Seed0_DataBostonHo...
4,BostonHousing,0,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.025,Classification,Seed0_DataBostonHousing_TP0.2_CP0.8_STTreeEnse...,BostonHousing/TreeFarms/Raw/Seed0_DataBostonHo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,BostonHousing,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.025,Classification,Seed99_DataBostonHousing_TP0.2_CP0.8_STTreeEns...,BostonHousing/TreeFarms/Raw/Seed99_DataBostonH...
596,BostonHousing,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,0,100,0.01,0.050,Classification,Seed99_DataBostonHousing_TP0.2_CP0.8_STTreeEns...,BostonHousing/TreeFarms/Raw/Seed99_DataBostonH...
597,BostonHousing,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.010,Classification,Seed99_DataBostonHousing_TP0.2_CP0.8_STTreeEns...,BostonHousing/TreeFarms/Raw/Seed99_DataBostonH...
598,BostonHousing,99,0.2,0.8,TreeEnsembleQBCFunction,TreeFarmsFunction,1,100,0.01,0.025,Classification,Seed99_DataBostonHousing_TP0.2_CP0.8_STTreeEns...,BostonHousing/TreeFarms/Raw/Seed99_DataBostonH...
